# YOLOv8 - Segmentação de Defeitos



## 1. Instalar Ultralytics

In [11]:
#!pip install ultralytics --upgrade -q

from ultralytics import YOLO
import os
import json
import cv2
from glob import glob
from tqdm import tqdm
import shutil
from sklearn.model_selection import train_test_split


## 2. Definir caminhos


In [12]:
base_dir = r'D:/blade_30_dataset_complete'
images_dir = os.path.join(base_dir, 'images')
annotations_dir = os.path.join(base_dir, 'annotations')

output_dir = r'D:/datasets/blade_stage2_defects'

## 3. Encontrar imagens com JSON correspondente


In [13]:
json_files = glob(os.path.join(annotations_dir, '*.json'))
image_json_pairs = []

for json_path in json_files:
    base_name = os.path.splitext(os.path.basename(json_path))[0]
    image_path = os.path.join(images_dir, f'{base_name}.jpg')
    if os.path.exists(image_path):
        image_json_pairs.append((image_path, json_path))
    else:
        print(f"[⚠️] JSON sem imagem correspondente: {base_name}")

print(f"✅ Total de pares imagem+json: {len(image_json_pairs)}")

[⚠️] JSON sem imagem correspondente: 13_116d39e3-2265-4c5f-9cdc-715628847fff
✅ Total de pares imagem+json: 260


## 4. Dividir treino/validação


In [14]:
train_pairs, val_pairs = train_test_split(image_json_pairs, test_size=0.2, random_state=42)


## 5. Função para converter JSON (polygon) para YOLOv8 .txt


In [15]:
# ========================
def convert_json_to_yolo(json_path, output_txt_path, image_w, image_h):
    with open(json_path, 'r') as f:
        data = json.load(f)

    with open(output_txt_path, 'w') as out:
        for shape in data['shapes']:
            label = shape['label']
            points = shape['points']
            if len(points) < 3:
                continue
            normalized_points = [f"{x/image_w:.6f} {y/image_h:.6f}" for x, y in points]
            out.write(f"0 {' '.join(normalized_points)}\n")  # classe 0 = defeito

## 6. Preparar estrutura de pastas


In [16]:
for split in ['train', 'val']:
    os.makedirs(os.path.join(output_dir, 'images', split), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'labels', split), exist_ok=True)


## 7. Processar imagens e anotações


In [17]:
def prepare_defect_split(pairs, split):
    for image_path, json_path in tqdm(pairs, desc=f'Processando {split}'):
        filename = os.path.basename(image_path)
        txt_name = os.path.splitext(filename)[0] + '.txt'

        img = cv2.imread(image_path)
        h, w = img.shape[:2]

        # Salvar imagem
        dst_img_path = os.path.join(output_dir, 'images', split, filename)
        shutil.copy(image_path, dst_img_path)

        # Converter JSON para .txt
        dst_lbl_path = os.path.join(output_dir, 'labels', split, txt_name)
        convert_json_to_yolo(json_path, dst_lbl_path, w, h)

prepare_defect_split(train_pairs, 'train')
prepare_defect_split(val_pairs, 'val')

Processando val: 100%|██████████| 52/52 [00:08<00:00,  5.88it/s]


## 8. Criar arquivo data.yaml


In [18]:
data_yaml = f"""
path: {output_dir}
train: images/train
val: images/val

nc: 1
names: ["defect"]
"""

with open(os.path.join(output_dir, 'defect.yaml'), 'w') as f:
    f.write(data_yaml)


## 9. Treinar modelo YOLOv8 para defeitos


In [19]:
# 9. Treinar modelo YOLOv8 para defeitos
# ========================
model = YOLO('yolov8l-seg.pt')
model.train(
    data=os.path.join(output_dir, 'defect.yaml'),
    epochs=85,
    imgsz=640,
    batch=8,
    name='blade_defect_segmentation'
)  # você pode trocar por yolov8s-seg.pt ou outro


New https://pypi.org/project/ultralytics/8.3.159 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.157  Python-3.12.0 torch-2.7.1+cpu CPU (13th Gen Intel Core(TM) i5-13450HX)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=D:/datasets/blade_stage2_defects\defect.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=85, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8l-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=blade_defect_segmentation

train: Scanning D:\datasets\blade_stage2_defects\labels\train.cache... 208 images, 0 backgrounds, 0 corrupt: 100%|██████████| 208/208 [00:00<?, ?it/s]

val: Fast image access  (ping: 0.10.0 ms, read: 420.472.4 MB/s, size: 3432.6 KB)



c:\Users\steva\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning D:\datasets\blade_stage2_defects\labels\val.cache... 52 images, 0 backgrounds, 0 corrupt: 100%|██████████| 52/52 [00:00<?, ?it/s]

Plotting labels to runs\segment\blade_defect_segmentation\labels.jpg... 


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):  50%|█████     | 2/4 [00:16<00:16,  8.28s/it]

WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:27<00:00,  6.77s/it]


                   all         52         74      0.785      0.593      0.673      0.367       0.67        0.5      0.537      0.241
Speed: 2.3ms preprocess, 378.1ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs\segment\blade_defect_segmentation


ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000284B72AE000>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.0410

## 10. ✅ Avaliando o modelo com .val()


In [20]:
# Avaliar com o conjunto de validação
metrics = model.val()
print(metrics)


Ultralytics 8.3.157  Python-3.12.0 torch-2.7.1+cpu CPU (13th Gen Intel Core(TM) i5-13450HX)
YOLOv8l-seg summary (fused): 125 layers, 45,912,659 parameters, 0 gradients, 220.1 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 539.654.4 MB/s, size: 4676.1 KB)


val: Scanning D:\datasets\blade_stage2_defects\labels\val.cache... 52 images, 0 backgrounds, 0 corrupt: 100%|██████████| 52/52 [00:00<?, ?it/s]
c:\Users\steva\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):   0%|          | 0/7 [00:00<?, ?it/s]

WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):  29%|██▊       | 2/7 [00:07<00:19,  3.95s/it]

WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:25<00:00,  3.69s/it]


                   all         52         74      0.785      0.593      0.673      0.367       0.67        0.5      0.537      0.241
Speed: 1.2ms preprocess, 358.1ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs\segment\blade_defect_segmentation2
ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000284B4E8B500>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,   

In [ ]:
# Avaliação após o treinamento
metrics = model.val()
print(f" mIoU: {metrics.metrics.segmentation.miou:.4f}")
print(f" mAP50: {metrics.metrics.segmentation.map50:.4f}")
print(f" Precision: {metrics.metrics.segmentation.precision:.4f}")
print(f" Recall: {metrics.metrics.segmentation.recall:.4f}")


NameError: name 'model' is not defined